In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sqlparse
import pandas as pd
from sqlalchemy import func, text

In [ ]:
from fiber.database.table import (
    fact,
    fd_diag,
    fd_proc,
    d_pers,
)
from fiber.database.hana import engine, Session
from fiber.cohort import Cohort
from fiber.clauses import Procedure, Diagnosis, BaseClause

In [ ]:
session = Session()

heart_surgery_cohort = Cohort(session,
    (
        Procedure('35.%') | Procedure('36.1%')
    ) &
    BaseClause(fact.AGE_IN_DAYS > 365 * 18)
#     BaseClause(fact.AGE_IN_DAYS > 365 * 18) &
#     Diagnosis('584.9')
)

aki_cohort = Cohort(session,
    Diagnosis('584.9')
)
aki_cohort.med_rec_numbers.count()

In [ ]:
print(sqlparse.format(str(heart_surgery_cohort.med_rec_numbers.statement), reindent=True))
number_of_patients = heart_surgery_cohort.med_rec_numbers.count()

print(f'Current number of qualifying patients {number_of_patients}')
print(f'Estimated total (0.9 bil of 2.4 bil facts ingested: {int(number_of_patients * 24/9)}')

In [ ]:
procedures = pd.read_sql(
    heart_surgery_cohort.query.with_entities(
        fd_proc.CONTEXT_PROCEDURE_CODE, fd_proc.PROCEDURE_DESCRIPTION, func.count(func.distinct(d_pers.MEDICAL_RECORD_NUMBER)).label('count')
    ).group_by(fd_proc.CONTEXT_PROCEDURE_CODE, fd_proc.PROCEDURE_DESCRIPTION).order_by(text('count DESC')).statement,
    engine
)


In [ ]:
procedures['label'] = procedures[['procedure_description', 'context_procedure_code']].apply(lambda x: f'{x[0]} ({x[1]})', axis=1)
ax = procedures[0:20].plot(x='label', y='count', kind="bar")
xlabels = ax.get_xticklabels()
ax.set_xticklabels(xlabels, rotation=40, ha='right')
title = ax.set_title(f'Top 20 procedures for {number_of_patients} patients')

In [ ]:
engine.execute('SET SCHEMA MSDW_2018')

aki = pd.read_sql(
    """
    SELECT fd.CONTEXT_DIAGNOSIS_CODE, COUNT(DISTINCT dp.MEDICAL_RECORD_NUMBER) count
    FROM FD_DIAGNOSIS fd 
    JOIN B_DIAGNOSIS b
        ON fd.DIAGNOSIS_KEY = b.DIAGNOSIS_KEY
    JOIN FACT2 f
        ON b.DIAGNOSIS_GROUP_KEY = f.DIAGNOSIS_GROUP_KEY
    JOIN D_PERSON dp
        ON f.PERSON_KEY = dp.PERSON_KEY
    WHERE f.PERSON_KEY IN (
        SELECT DISTINCT dp.PERSON_KEY
        FROM FD_PROCEDURE fd 
        JOIN B_PROCEDURE b
         ON fd.PROCEDURE_KEY = b.PROCEDURE_KEY
        JOIN FACT2 f
            ON b.PROCEDURE_GROUP_KEY = f.PROCEDURE_GROUP_KEY
        JOIN D_PERSON dp
            ON f.PERSON_KEY = dp.PERSON_KEY
        WHERE (
            fd.CONTEXT_PROCEDURE_CODE LIKE '35.%'
            OR fd.CONTEXT_PROCEDURE_CODE LIKE '36.1%'
        )
        AND f.AGE_IN_DAYS > 6570
    )
    AND (
        fd.CONTEXT_DIAGNOSIS_CODE = '584.9'
        OR fd.CONTEXT_DIAGNOSIS_CODE LIKE '433%'
        OR fd.CONTEXT_DIAGNOSIS_CODE LIKE '434%'
        OR fd.CONTEXT_DIAGNOSIS_CODE LIKE '436%'
    )
    AND fd.CONTEXT_NAME = 'ICD-9'
    GROUP BY fd.CONTEXT_DIAGNOSIS_CODE
    ORDER BY count DESC
    """,
    engine
)

In [ ]:
# aki['label'] = aki[['description', 'context_diagnosis_code']].apply(lambda x: f'{x[0]} ({x[1]})', axis=1)
ax = aki[0:20].plot(x='context_diagnosis_code', y='count', kind="bar")
xlabels = ax.get_xticklabels()
ax.set_xticklabels(xlabels, rotation=40, ha='right')
title = ax.set_title(f'Top comorbidities for {number_of_patients} patients and ICD-9 like (584.9, 433.*, 434.*, 435.*)')

In [ ]:
glucose = pd.read_sql(
    '''
    SELECT dp.MEDICAL_RECORD_NUMBER, f.VALUE
    FROM FD_PROCEDURE fd 
    JOIN B_PROCEDURE b
        ON fd.PROCEDURE_KEY = b.PROCEDURE_KEY
    JOIN FACT2 f
        ON b.PROCEDURE_GROUP_KEY = f.PROCEDURE_GROUP_KEY
    JOIN D_PERSON dp
        ON f.PERSON_KEY = dp.PERSON_KEY
    WHERE f.PERSON_KEY IN (
        SELECT DISTINCT dp.PERSON_KEY
        FROM FD_PROCEDURE fd 
        JOIN B_PROCEDURE b
         ON fd.PROCEDURE_KEY = b.PROCEDURE_KEY
        JOIN FACT2 f
            ON b.PROCEDURE_GROUP_KEY = f.PROCEDURE_GROUP_KEY
        JOIN D_PERSON dp
            ON f.PERSON_KEY = dp.PERSON_KEY
        WHERE (
            fd.CONTEXT_PROCEDURE_CODE LIKE '35.%'
            OR fd.CONTEXT_PROCEDURE_CODE LIKE '36.1%'
        )
        AND f.AGE_IN_DAYS > 6570
    )
    AND fd.PROCEDURE_DESCRIPTION = 'Glucose; blood, reagent strip'
    ''',
    engine
)

In [ ]:
class OrTest:
    def __init__(self, name):
        self.name = name
    
    def __or__(self, other):
        print(self.name, other.name)
        return OrTest(f'{self.name}OR{other.name}')

In [ ]:
(
    OrTest('OuterLeft') | (
        OrTest(1) | (
            OrTest(2.1) | OrTest(2.2)
        )
    ) | OrTest('OuterRight')
)